In [ ]:
#导入所需的包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import NullFormatter 

# 蒙特卡洛模拟
N = 1000  # 1000名玩家
T = 100  # 下注次数为100次
S_0 = 100  # 资金初始值
w = 0.5  # 赢了赚50%
l = 0.4  # 亏了赔40%

# 用正态分布模拟，构造随机的收益率分布，（1000，100）
w_l = pd.DataFrame(np.where(np.random.randn(N,T)>0, 1+w, 1-l))

#初始的资产
p = pd.DataFrame((np.zeros((N,1)) + S_0))

#初始资产放入收益率序列，累乘用
w_l.insert(0,'p_0',p)

#每一次下注后的资产，按行累乘
w_l_T = w_l.cumprod(axis = 1)

#最终的资产数量及分布概览
p_T = w_l_T[99]
print(p_T.describe())

#计算几何平均值和算术平均值
geo_average = p_T.median()
arith_average = p_T.mean()

# 以10为底，对最终资产进行对数化处理
geo_average_log = np.log10(p_T.median())
arith_average_log = np.log10(p_T.mean())

#以e为底，对最终资产进行对数化处理
geo_average_loge = np.log(p_T.median())
arith_average_loge = np.log(p_T.mean())

#对资产进行对数化处理，绘制每一步下注后的资产变化曲线
w_l_T_log = np.log(w_l_T)
#for i in range(101):
    #w_l_T_log.loc[i].plot()

#以10为底，对资产进行对数化处理，绘制每一步下注后的资产变化曲线
w_l_T_log10 = np.log10(w_l_T)
#for i in range(101):
    #w_l_T_log10.loc[i].plot()

#绘制最终资产分布直方图
plt.figure(figsize =(8,4))
sns.histplot(p_T,bins=100,color='r',label='wealth')
plt.legend()

#绘制最终资产分布直方图,以10为底
plt.figure(figsize =(8,4))
sns.histplot(p_T,bins=100,color='r',label='wealth',log_scale = 10)
plt.legend()

#绘制资产变化路径及最终分布
# 规划一下两个坐标轴的位置
left, width = 0.1, 0.7
bottom, height = 0.1, 0.7
left_h = left + width + 0.02

rect_plot = [left, bottom, width, height]  # 折线图的边界
rect_histy = [left_h, bottom, 0.25, height]  # 直方图的边界

# 初始化图像
plt.figure(1, figsize=(8, 8))

# 定义两个画图区域，获得相应的handle
axPlot = plt.axes(rect_plot)  # 折线图的区域
axHisty = plt.axes(rect_histy)   # 直方图区域

# 折线图的横坐标，即模拟时间，这里需要 T+1 ，因为添加了初值
Ts = list(range(T+1))

# 直方图不需要轴标签 no labels
#nullfmt = NullFormatter()
#axHisty.yaxis.set_major_formatter(nullfmt)

# 画出折线图
for i in range(N): # 这里不用 N+1 ，因为只有 N 次模拟
    axPlot.plot(Ts,w_l_T_log10.loc[i,:],linewidth=1, color='gray', linestyle='-')  # 选用以10为底，对数化之后的数据
n, _, _ = axHisty.hist(np.log10(p_T), bins=30, orientation='horizontal')  # 直方图，同样需要以10为底，对数化之后的数据
n = (max(n) // 5) * 5 
axHisty.plot([0, n], [geo_average_log, geo_average_log], label='median', linewidth=5, color='red', linestyle='--')  # 添加中位数，取对数后值
axHisty.plot([0, n], [arith_average_log, arith_average_log], label='mean', linewidth=5, color='green', linestyle=':')  # 添加均值，取对数值
axHisty.legend()
axHisty.set_ylim(axPlot.get_ylim())

plt.show()

count      1000.000000
mean        697.675490
std        7139.016282
min           0.000001
25%           0.032984
50%           0.515378
75%          20.131934
max      191993.087803
Name: 99, dtype: float64
